In [0]:
from pyspark.sql.functions import first, rand, count
dates = spark.sql("SELECT explode(sequence(DATE'2024-01-01', DATE'2024-03-24', INTERVAL 1 DAY)) as  calendar_date")
c_id = spark.sql("SELECT explode(sequence(1,12500, 1)) as  client_id")
types = spark.sql("""SELECT concat("col_", colName) as col_name from (SELECT explode(sequence(1,100, 1)) as  colName)""")
 
dates = dates.repartition(99)
c_id = c_id.repartition(11)
types = types.repartition(1)
 
df_cartesian = c_id.crossJoin(dates.select("calendar_date")).crossJoin(types.select("col_name")).select("client_id","calendar_date","col_name")
df_cartesian2 = df_cartesian.groupBy("calendar_date").agg(count("client_id"))
 
# display(df_cartesian2.limit(1000))
 
df_cartesian = df_cartesian.withColumn("val", (rand()*10).cast("int"))

df_grp1 = df_cartesian.groupBy("client_id","calendar_date").pivot("col_name").agg((first("val").alias("val")))


types2 = spark.sql("SELECT concat('B_col_', colName) as col_name FROM (SELECT explode(sequence(1,50, 1)) as colName)")

df_cartesian2 = c_id.crossJoin(dates).crossJoin(types2).select("client_id", "calendar_date", "col_name")
df_cartesian2 = df_cartesian2.withColumn("val", (rand() * 20).cast("int")) 
df_grp2 = df_cartesian2.groupBy("client_id", "calendar_date").pivot("col_name").agg(first("val"))


#display(df_grp)
print(f"Table 1 Count: {df_grp2.count()}")


Table 1 Count: 1050000


In [0]:
df_inner = df_grp1.join(df_grp2, ["client_id", "calendar_date"], "inner")
#display(df_inner)

In [0]:
df_left = df_grp1.join(df_grp2, ["client_id", "calendar_date"], "left")

In [0]:
df_inner_no_dupes = df_inner.select("client_id", "calendar_date")
df_inner_no_dupes.show()

+---------+-------------+
|client_id|calendar_date|
+---------+-------------+
|    10922|   2024-01-07|
|     8090|   2024-03-16|
|     6607|   2024-02-09|
|     4715|   2024-03-21|
|     4715|   2024-02-03|
|     8370|   2024-02-20|
|      999|   2024-02-27|
|      999|   2024-02-21|
|    11398|   2024-02-11|
|     3832|   2024-03-16|
|     3832|   2024-02-11|
|     3832|   2024-02-05|
|     5121|   2024-02-23|
|     6798|   2024-03-18|
|     2430|   2024-03-04|
|    11911|   2024-02-01|
|    10638|   2024-03-18|
|     8632|   2024-02-13|
|    10246|   2024-02-10|
|    10246|   2024-02-25|
+---------+-------------+
only showing top 20 rows



In [0]:
df_left_no_dupes = df_left.select("client_id", "calendar_date")